In [1]:
# 窗口变化位置win_change，上升沿标记top_win
def draw_rising_edge(data,win_change,top_win):
    flag = True
    for i in range(top_win.shape[0]-1): 
        if top_win[i] == 1 and top_win[i+1]==0:
            if flag:
                plt.plot(range(win_change[i],win_change[i+1]),data[win_change[i]:win_change[i+1]],c='r',lw=1.2)
                flag=False
            else:
                plt.plot(range(win_change[i],win_change[i+1]),data[win_change[i]:win_change[i+1]],c='r',lw=1.2)

In [1]:
# 绘制4行2列图形【原始波形，曼哈顿，pca降维，均值波形，单调性分界】
def plot_x_y(orignal_x,orignal_y,x,y,cnn,mean,
             win_change,
             win_size,step,top_win,
             mean_count=100,delta=5,count=100):
    plt.figure(figsize=(30,16),dpi=800)
    font = FontProperties(family='Times New Roman')
    plt.subplot(4,2,1)
    plt.plot(range(len(orignal_x)),orignal_x,color='g',lw=0.5,label='orignal_x')
    plt.legend(loc=0,prop=font)
    plt.title('初始x波形')
    plt.subplot(4,2,2)
    plt.plot(range(len(orignal_y)),orignal_y,color='#FF6A6A',label='orignal_y',lw=0.1)
    plt.legend(loc=0,prop=font)
    plt.title('初始y波形')
    
    plt.subplot(4,2,3)
    plt.plot(range(len(x)),x,color='r',lw=0.5,label='x')
    plt.legend(loc=0,prop=font)
    plt.title(f'Δ={delta},count={count}距离压缩后x波形')
    plt.subplot(4,2,4)
    plt.plot(range(len(y)),y,label='y',lw=0.1)
    plt.legend(loc=0,prop=font)
    plt.title(f'Δ={delta},count={count}距离压缩后y波形')
    
    # PCA 之后波形
    plt.subplot(4,2,5)
    plt.plot(range(len(cnn)),cnn,lw=1,label='pca')
    plt.legend(loc=0,prop=font)
    plt.title(f'PCA 后的波形')
    
    plt.subplot(4,2,6)
    plt.plot(range(len(mean)),mean,color='#00CCFF',label='mean',lw=1)
    plt.legend(loc=0,prop=font)
    plt.title(f'每{mean_count}个点均值压缩后的波形')
    
    plt.subplot(4,2,7)
    plt.plot(range(len(mean)),mean,lw=1,label='mean')
    draw_rising_edge(mean,win_change,top_win)
#     设置图例字体
    plt.legend(loc=0,prop=font)
    plt.title(f'win_size_x={win_size},step_x={step}选取单调性分界点情况')
    
    plt.show()
    return x,y

In [4]:
def plt_all(file_name,class_num=-1,nature_flag=True):
    df = None
    df1 = None
    df2 = None
    if nature_flag:
        # 原始数据
#         df = pd.read_csv(f'../../event_csv/split_data/class{class_num}/{file_name}',skiprows=1)
        df = pd.read_csv(f'../../event_csv/split_data/class{class_num}/{file_name}')
        # 曼哈顿距离
        df1 = pd.read_csv(f'../../event_csv/compress_event_manhattan/class{class_num}/{file_name}')
        # PCA 后
        df2 = pd.read_csv(f'../../event_csv/compress_event_manhattan/class{class_num}/smooth_by_pca/{file_name}')
        # 取均值后
        df3 = pd.read_csv(f'../../event_csv/compress_event_manhattan/class{class_num}/smooth_by_pca/compress_by_mean/{file_name}')
    else:
        # 人工合成数据
        df = pd.read_csv(f'../../event_csv/split_data/artificial/{file_name}')
        # 曼哈顿距离
        df1 = pd.read_csv(f'../../event_csv/compress_event_manhattan/articicial/{file_name}')
        # 取均值后
        df2 = pd.read_csv(f'../../event_csv/compress_event_manhattan/articicial/compress_by_mean/{file_name}')
        
    win_change,top_win,win_size,step = get_index_of_bottom_and_top_by_mk(df3['value'])
    x,y = plot_x_y(df['x'],df['y'],df1['x'],df1['y'],df2['value'],df3['value'],
             win_change,
             win_size,step,
             top_win)
    return x,y

In [8]:
# 拆分df中的各列
def get_x_y_t(df):
    # 时间戳顺序展开
    xs = np.arange(df.shape[0])
    ys = df['y']
    zs = df['x']
    return xs,ys,zs
def set_label(ax):
    ax.set_xlabel('T',fontsize=20)
    ax.set_ylabel('Y',fontsize=20)
    ax.set_zlabel('X',fontsize=20,labelpad=0)

In [ ]:
 # 绘制3D图形
def plot_3D(file_name,class_num=None,delta=5,count=100,mean_count=100):
    fig = plt.figure(figsize=(23,12),dpi=100)
    font = FontProperties(family='Times New Roman',size=20)
    ax = fig.add_subplot(121, projection='3d')
    # 原始3D图
    df = pd.read_csv(f'../../event_csv/split_data/class{class_num}/{file_name}')
    xs,ys,zs = get_x_y_t(df)
    ax.scatter(xs, ys, zs,s=2,c='blue',label='Event')
    plt.title(f'Original event stream',size=20)
    set_label(ax)
    
    ax = fig.add_subplot(122, projection='3d') 
    # 距离压缩后的3D图
    df = pd.read_csv(f'../../event_csv/compress_event_manhattan/class{class_num}/{file_name}')
    xs,ys,zs = get_x_y_t(df)
    ax.scatter(xs, ys, zs, s=2,c='red',label='Event')
    plt.title(f'Event stream is processed by Δ={delta},count={count}',size=20)
    set_label(ax)
    plt.show()

In [ ]:
def plt_2D(file_name,class_num,mean_count=100):
    plt.figure(figsize=(26,8),dpi=100)
    # PCA 之后波形
    plt.subplot(2,2,1)
    df = pd.read_csv(f'../../event_csv/compress_event_manhattan/class{class_num}/smooth_by_pca/{file_name}')
    plt.plot(np.arange(df.shape[0]),df['value'],lw=1,label='pca')
    plt.title(f'PCA 后的波形')
    
    plt.subplot(2,2,2)
    df = pd.read_csv(f'../../event_csv/compress_event_manhattan/class{class_num}/smooth_by_pca/compress_by_mean/{file_name}')
    plt.plot(np.arange(df.shape[0]),df['value'],color='#00CCFF',label='mean',lw=1)
    plt.title(f'每{mean_count}个点均值压缩后的波形')
    
    plt.subplot(2,2,3)
    plt.plot(np.arange(df.shape[0]),df['value'],lw=1,label='mean')
    win_change,top_win,win_size,step = get_index_of_bottom_and_top_by_mk(df['value'])
    draw_rising_edge(df['value'],win_change,top_win)
#     设置图例字体
#     plt.legend(loc=0,prop=font)
    plt.title(f'win_size_x={win_size},step_x={step}选取单调性分界点情况')
    plt.show()